In [1]:
from fastai.vision import *
from fastai.callbacks import *
from pytorchcv.model_provider import get_model as ptcv_get_model
from torchvision.models import *
import pretrainedmodels
from pathlib import Path
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from tqdm import tqdm


In [2]:
def efficientnet_b0(pretrained=False):
    return ptcv_get_model("efficientnet_b0", pretrained=True).features

def efficientnet_b1(pretrained=False):
    return ptcv_get_model("efficientnet_b1", pretrained=True).features

def resnext50(pretrained=False):
    return ptcv_get_model("resnext50_32x4d", pretrained=True).features

def resnext26(pretrained=False):
    return ptcv_get_model("resnext26_32x4d", pretrained=True).features

In [3]:
def add_learners(model):
    np.random.seed(2)
    data = ImageDataBunch.from_folder(path,size=model_params[model]['size'], bs=model_params[model]['bs'])
    learn = cnn_learner(data, model_params[model]['model'], pretrained=False,
                    cut=noop, metrics=metrics, split_on=lambda m: (m[0][4], m[1]))
    learn.load(model_params[model]['weights'])
    return learn

In [4]:
def get_missing_metrics(ENSAMBLE_LEARN):
    for learn in list(zip(['RESNEXT50', 'EFFICIENT0', 'EFFICIENT1'], ENSAMBLE_LEARN[1:])):
        val_preds, val_targs = learn[1].get_preds()
        val_preds = torch.argmax(val_preds, axis=1)
        print(learn[0], 'ACCURACY =', accuracy_score(val_targs, val_preds), 'F1 =', f1_score(val_targs,val_preds,average='macro'))

In [5]:
def get_targets(learn):
    _, targs = ENSAMBLE_LEARN[0].get_preds()
    return targs

In [6]:
def calculate_ensamble(ENSAMBLE_LEARN,targs):
    preds = torch.zeros(2000,7)
    for learn in ENSAMBLE_LEARN:
        val_preds, _ = learn.get_preds()
        preds += val_preds
    preds /= len(ENSAMBLE_LEARN)
    predictions = torch.argmax(preds, axis=1)
    acc = accuracy_score(targs, predictions)
    f1 = f1_score(targs,predictions,average='macro')
    b_acc = balanced_accuracy_score(targs, predictions)
    print('ACCURACY', acc, 'F1', f1, 'Balanced Accuracy' , b_acc)

In [7]:
metrics=[FBeta('macro', beta=1), accuracy]
MODELS = ['resnext26', 'resnext50','efficient0', 'efficient1']
ENSAMBLE_LEARN = []       
path = Path("./data") 
model_params = {
    'resnext50': {
        'size': 299,
        'bs': 10,
        'model': resnext50,
        'weights':'resnext50_notpr-0'
    },
    'resnext26': {
        'size': 299,
        'bs': 10,
        'model': resnext26,
        'weights':'best_model_nexttest-1'
    },
    'efficient0': {
        'size': 448,
        'bs': 10,
        'model': efficientnet_b0,
        'weights': 'best_model_effb0pr-0'
    },
    'efficient1': {
        'size': 448,
        'bs': 10,
        'model': efficientnet_b1,
        'weights': 'best_model_effb1-1'
    }
}


In [8]:
for m in MODELS:
    ENSAMBLE_LEARN.append(add_learners(m))

In [9]:
targs =get_targets(ENSAMBLE_LEARN[0])

In [10]:
# For displaying the seperate metrices of models other than resnext26
# (results of resnext26 are provided in Train.ipynb)
get_missing_metrics(ENSAMBLE_LEARN)

RESNEXT50 ACCURACY = 0.9125 F1 = 0.8791974447472556


EFFICIENT0 ACCURACY = 0.9135 F1 = 0.8686517346015304


EFFICIENT1 ACCURACY = 0.9005 F1 = 0.8515617588302921


In [11]:
calculate_ensamble(ENSAMBLE_LEARN, targs)

ACCURACY 0.9275 F1 0.9010500832233845 Balanced Accuracy 0.8786365064439164
